In [1]:
# Import Dependencies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
import datetime

# Cleaning data

## Clean forex data

In [ ]:
# csv directory
file = '../data/forex_dta.csv'

# read csv
forex = pd.read_csv(file)

# list of months
months = ["Jan", "Feb", "Mar", "Apr", "May", "June", "July", "August", "Sept", "Oct", "Nov", "Dec"]

# slice the "Rate" column and create new columns for the currency (three letter initials) 
# and current rate (with relation to EUR)
forex["currency"] = forex["Rate"].str.slice(2, 5, 1) 
forex["rate"] = forex["Rate"].str.slice(8,-1, 1)

# slice date and create columns for year and month
forex["year"] = forex["Date"].str.slice(0, 4, 1) 
forex["month"] = forex["Date"].str.slice(5,7, 1)
forex["month"] = pd.to_numeric(forex["month"])
forex["month_str"] = ""

for i in range(480):
    forex["month_str"][i] = months[forex["month"][i] - 1]

# export to csv
forex.to_csv(r'../data_cleaning/forex_data.csv')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Clean expenditures

In [ ]:
file = "../data_cleaning/travel_expenditures.csv"

df = pd.read_csv(file, header=None)
df.dropna(axis=1)
for x in range(120,132):
    del df[x]
df.head()

In [ ]:
df = df.transpose()
df.head()

In [ ]:
df["Year"] = pd.DatetimeIndex(df[0]).year
df["Month"] = pd.DatetimeIndex(df[0]).month
df.columns = ["Date", "Expenditures", "Year", "Month"]
df.head()

In [ ]:
df.to_csv("../data_cleaning/cleaned_travel_expenditures.csv")

# Plots

## Currency rates

In [ ]:
# Load file for currency
currency = pd.read_csv("../data_cleaning/forex_data.csv")

currency.head()

In [ ]:
#rename Rate to country name and GDP
del currency['Unnamed: 0.1']
currency_2 = currency
currency_2.head()

In [ ]:
currency_3 = currency_2.pivot(index="Date", columns="currency")['rate']
currency_3.head()

currency_3.index = pd.to_datetime(currency_3.index)

currency_3.index = currency_3.index.strftime('%Y-%m')

currency_3.head()

In [ ]:
from matplotlib.ticker import MaxNLocator
locator=MaxNLocator(prune='both', nbins=30)

In [ ]:
# Set parameters for plot for JPY
plt.figure(figsize=(10,2))

# Plot
plt.plot(currency_3.index, currency_3["JPY"], color="black",markersize=1, linestyle="-", linewidth=2.0)

# Set title, x and y axises, and add grids and legends
plt.title("Yen Over Time")
plt.ylabel("Rates", fontsize = 14 )
plt.yticks(fontsize = 12)
plt.xlabel("Date", fontsize = 16)
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.legend(['YEN'],loc="best", fontsize="medium", fancybox=True)
ax = plt.gca()
ax.xaxis.set_major_locator(locator)

# Save plot
plt.savefig("../graphs/yen.png")

# ax.set_xlim([])
#ax.set_ylim([0,70])

In [ ]:
# Do the same for USD
plt.figure(figsize=(10, 2))
plt.plot(currency_3.index, currency_3["USD"], color="green", markersize=1, linestyle="-", linewidth=2.0)
plt.title("USD Over Time")
plt.ylabel("Rates", fontsize = 14)
plt.yticks(fontsize = 12)
plt.xlabel("Date")
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.legend(['USD'],loc="best", fontsize="medium", fancybox=True)
ax = plt.gca()
ax.xaxis.set_major_locator(locator)
plt.savefig("../graphs/usd.png")

In [ ]:
# Do the same for CNY
plt.figure(figsize=(10, 2))
plt.plot(currency_3.index, currency_3["CNY"], color="red", markersize=1, linestyle="-", linewidth=2.0)
plt.title("CNY Over Time")
plt.ylabel("Rates", fontsize = 14)
plt.yticks(fontsize = 12)
plt.xlabel("Dates")
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.legend(['CNY'],loc="best", fontsize="medium", fancybox=True)
ax = plt.gca()
ax.xaxis.set_major_locator(locator)
plt.savefig("../graphs/cny.png")

In [ ]:
# Do the same for GBP
plt.figure(figsize=(10, 2))
plt.plot(currency_3.index, currency_3["GBP"], color="blue", markersize=1, linestyle="-", linewidth=2.0)
plt.title("Pound Over Time")
plt.ylabel("Rates", fontsize = 14)
plt.yticks(fontsize = 12)
plt.xlabel("Date")
plt.grid(axis='y')
plt.xticks(rotation=45)
plt.legend(['GBP'],loc="best", fontsize="medium", fancybox=True)
ax = plt.gca()
ax.xaxis.set_major_locator(locator)
plt.savefig("../graphs/gbp.png")

## Currency vs. Travel in and Travel out in the US

In [ ]:
# Import currency and travel csv
data_path = "../data_cleaning/"
currency_data = pd.read_csv(data_path + "forex_data.csv")
travel_data = pd.read_csv(data_path + "cleaned_travel_expenditures.csv")
trip_data = pd.read_csv(data_path + "trips.csv")

# Format date in yyyy-mm-dd
travel_data["Date"] = pd.to_datetime(travel_data["Date"])
currency_data["Date"] = pd.to_datetime(currency_data["Date"])
trip_data["Date"] = pd.to_datetime(trip_data["Date"])

travel_data = travel_data.drop(["Unnamed: 0"], axis=1)
currency_data = currency_data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
# currency_data = currency_data.set_index("currency")

usa_currency = currency_data.loc[currency_data['currency'] == 'USD']

combined_data = pd.merge(travel_data, usa_currency, on=["Date"])
combined_data = pd.merge(combined_data, trip_data, on=["Date"])

combined_data.head()

In [ ]:
# Set x and y axises
x_axis = combined_data["rate"]
departure = combined_data["outbound"]
departure = departure.str.replace(',', '')
departure = departure.astype(float)

# Plot scatter graph for currency rate vs. travel expenditure
plt.scatter(x_axis, departure, edgecolor="black")

# Set plot title, x-axis and y-axis labels
plt.title("Currency Rates vs. Traveling from the USA (Corr=0.02,p=0.84)")
plt.xlabel("Currency Rate (to EUR)")
plt.ylabel("Travel Outbound")

# Add grids
plt.grid()

# Save figure
plt.savefig("../graphs/currency vs. travel outbound.png")

# Show the chart
plt.show()

In [ ]:
# Calculate the correlation between USD currency rate and travel outbound
np.corrcoef(x_axis, departure)

In [ ]:
# Regression: dependent variable is departure and independent variable is currency rate
linregress(x_axis, departure)

In [ ]:
# Set x and y axises
arrival = combined_data["inbound"]
arrival = arrival.str.replace(',', '')
arrival = arrival.astype(float)

# Plot scatter graph for currency rate vs. travel expenditure
plt.scatter(x_axis, arrival, edgecolor="black", c= "green")

# Set plot title, x-axis and y-axis labels
plt.title("Currency Rates vs. Traveling to the USA (Corr=0.20, p<0.05)")
plt.xlabel("Currency Rate (to EUR)")
plt.ylabel("Travel Inbound")

# Add grids
plt.grid()

# Save figure
plt.savefig("../graphs/currency vs. travel inbound.png")

# Show the chart
plt.show()

In [ ]:
# Calculate the correlation between USD currency rate and travel inbound
np.corrcoef(x_axis, arrival)

In [ ]:
# Regression: dependent variable is arrival and independent variable is currency rate
linregress(x_axis, arrival)

In [ ]:
# Find the slope and interception of the best fit line
def best_fit(X, Y):

    xbar = sum(X)/len(X)
    ybar = sum(Y)/len(Y)
    n = len(X) # or len(Y)

    numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
    denum = sum([xi**2 for xi in X]) - n * xbar**2

    b = numer / denum
    a = ybar - b * xbar

    print('best fit line:\ny = {:.2f} + {:.2f}x'.format(a, b))

    return a, b

# solution
a, b = best_fit(x_axis, arrival)

# plot points and fit line
plt.scatter(x_axis, arrival)
yfit = [a + b * xi for xi in x_axis.sort_values()]
plt.plot(x_axis.sort_values(), yfit)